Company:
LinkedIn
◆
Difficulty:
Hard
[Question Link](https://www.interviewmaster.ai/question/feed-content-quality-engagement-and-insights-impact)

As a Product Analyst on the LinkedIn Feed team, you are tasked with analyzing how different content types impact user engagement on the platform. Your team is focused on understanding which types of content consistently drive higher engagement scores. The ultimate goal is to leverage these insights to enhance content recommendation strategies and improve user experience on the feed.

**Question 1 of 3**

Identify which content formats performed best in July 2024 by reporting their average engagement scores, but only list the formats that achieved an average score of 50 or higher.

In [ ]:
#This question wanna know avg(engagement_score) per content_type
#and that average must greater or equal than 50
#by use engagement_date cuz 'content formats performed'
SELECT content_type,
  AVG(engagement_score) AS avg_engage_scores
FROM fct_user_engagement f
JOIN dim_content d ON
  f.content_id = d.content_id
WHERE engagement_date BETWEEN
  '2024-07-01' AND '2024-07-31'
GROUP BY 1
HAVING AVG(engagement_score) >= 50
ORDER BY 2 DESC;

**Question 2 of 3**

For the first week of August 2024, identify the content type of the content that had the highest engagement score. For that content type, calculate the average engagement score for the entire month of August. If there is a tie for highest engagement score, select the content type with the earliest publish date.

In [ ]:
#This question wanna know avg_engagement_score in Aug 2024 but depend on
#first week aug 2024 what content_type are highest score
#if have more than one are same show onlu earliest publish date
WITH engagement_august AS(
  SELECT d.content_type AS contents_type,
    f.engagement_score AS engagement_score,
    f.engagement_date AS engagement_date,
    d.publish_date AS publish_date
  FROM fct_user_engagement f
  JOIN dim_content d ON
    f.content_id =d.content_id
  WHERE f.engagement_date BETWEEN '2024-08-01' AND '2024-08-31'
),
first_week_engagement AS(
  SELECT contents_type,
    engagement_score,
    publish_date
  FROM engagement_august
  WHERE engagement_date BETWEEN
    '2024-08-01' AND '2024-08-07'
),
content_highest_score AS (
  SELECT contents_type
  FROM first_week_engagement
  ORDER BY engagement_score DESC, publish_date
  LIMIT 1
)
SELECT contents_type,
  AVG(engagement_score) AS avg_engagement_scores
FROM engagement_august
WHERE contents_type IN (SELECT contents_type FROM content_highest_score)
GROUP BY 1;

**Question 3 of 3**

During Q3 2024, for each content type, calculate their highest engagement score each week. What was the average of those weekly highs over the quarter? This will help us identify content types that consistently generate peak engagement.

In [ ]:
#This question wanna know in Q3/2024(July-September) each content_type
#highest_engage_Score per week and find avg_engagge from max previous
#per content_type in Q3
WITH highest_score_each_week AS(
  SELECT EXTRACT(WEEK FROM engagement_date) AS weekly,
    content_type,
    MAX(engagement_score) AS highest_engage_scores
  FROM fct_user_engagement f
  JOIN dim_content d ON
    f.content_id = d.content_id
  WHERE engagement_date BETWEEN '2024-07-01' AND '2024-09-30'
  GROUP BY 1, 2
),
avg_max_score AS(
  SELECT content_type,
    AVG(highest_engage_scores) AS avg_highest_engage_scores
  FROM highest_score_each_week
  GROUP BY 1
)
SELECT *
FROM avg_max_score;